In [1]:
from     bs4 import  BeautifulSoup as soup
import   requests
import   pandas as pd
from     urllib.request  import  urlopen
import  xml.etree.ElementTree  as ET


# Create list for Maker’s home page URL
makers_List = ['https://www.kbb.com/acura/' , 'https://www.kbb.com/alfa-romeo/' , 'https://www.kbb.com/aston-martin/' , 'https://www.kbb.com/audi/' , 'https://www.kbb.com/bentley/',  'https://www.kbb.com/bmw/' , 'https://www.kbb.com/buick/' , 'https://www.kbb.com/cadillac/' 'https://www.kbb.com/chevrolet/' , 'https://www.kbb.com/chrysler/' , 'https://www.kbb.com/dodge/' , 'https://www.kbb.com/ferrari/' , 'https://www.kbb.com/fiat/' , 'https://www.kbb.com/ford/' , 'https://www.kbb.com/freightliner/' , 'https://www.kbb.com/genesis/' , 'https://www.kbb.com/gmc/' , 'https://www.kbb.com/honda/' , 'https://www.kbb.com/hyundai/' , 'https://www.kbb.com/infiniti/' , 'https://www.kbb.com/jaguar/' , 'https://www.kbb.com/jeep/' , 'https://www.kbb.com/kia/' , 'https://www.kbb.com/lamborghini/' , 'https://www.kbb.com/land-rover/',  'https://www.kbb.com/lexus/' , 'https://www.kbb.com/lincoln/' , 'https://www.kbb.com/maserati/' , 'https://www.kbb.com/mazda/' , 'https://www.kbb.com/mclaren/' , 'https://www.kbb.com/mercedes-benz/' , 'https://www.kbb.com/mini/' , 'https://www.kbb.com/mitsubishi/' , 'https://www.kbb.com/nissan/' , 'https://www.kbb.com/porsche/', 'https://www.kbb.com/ram/' , 'https://www.kbb.com/rivian/' , 'https://www.kbb.com/rolls-royce/' , 'https://www.kbb.com/smart/' , 'https://www.kbb.com/subaru/' , 'https://www.kbb.com/tesla/' , 'https://www.kbb.com/toyota/' , 'https://www.kbb.com/volkswagen/' , 'https://www.kbb.com/volvo/']

# Create list for annual inflation rate in the USA  (2020 - 1964)
# URL: https://www.usinflationcalculator.com/inflation/historical-inflation-rates/  
annual_inflation_rate_List = [0 , 0.018, 0.024, 0.021, 0.013, 0.001, 0.016, 0.015, 0.021, 0.032, 0.016, -0.004, 0.038, 0.028, 0.032, 0.034, 0.027, 0.023, 0.016, 0.028, 0.034, 0.022, 0.016, 0.023, 0.03, 0.028, 0.026, 0.03, 0.042, 0.054, 0.048, 0.041, 0.036, 0.019, 0.036, 0.043, 0.032, 0.062, 0.103, 0.135, 0.113, 0.076, 0.065, 0.058, 0.091, 0.11, 0.062, 0.032, 0.044, 0.057, 0.055, 0.042, 0.031, 0.029, 0.016, 0.013, 0.013]

mListSize = len(makers_List)
print ('Maker List size = ' , mListSize)
print("\n\n")
prop = {}
tmp_dct = {}
links = []
Maker = ''
first_year = ''
i = 0
m = []
values = []
model_dct_list = []                                       # list of all models for a given maker
tmp_dct_list = []                                         # list of dictionaries structure used by called functions.
model_years_dct_list = []                                 # holds detailed model info (model, first-year, price, model_url>
model_final_dct_list = []                                 # final including all years for given model

# input:  maker_url
# output: {Maker : maker, Model : v1,  First Year : v2,  Price : v3,  url : link}, {}….] }
def  get_maker_models(maker_url) : 
    r3 = requests.get(maker_url)
    soup1 = soup( r3.content, 'html.parser')

    # find iterative top level TAG for each maker/model
    #link_as = soup1.find('div', {"class":"make-list-wrapper"}).find_all('a', {"class" : "js-ymm-link result-ymm-title js.sponsored-url"}, href=True)
    link_as = soup1.find('div', {"class":"make-list-wrapper"}).find_all('div', {"class" : "results-row js-results-row"})
    
    i = 0
    for   y  in   link_as :                                           # y is the top-level TAG for 1st year models
        # from that top level get: links, maker, model, first-year for all top-year Models for that Maker’s new Cars
        #x = y.find('a', {"class" : "js-ymm-link result-ymm-title js.sponsored-url"}, href=True)
        x = y.find('a', href=True)
        link = "https://www.kbb.com" + x['href']                       # x['href'] has '/Maker/Model'
        if link not in links:
            links.append(link)
            maker, model = (x['href'].strip('/')).split('/')
            Maker = maker[:]
            # globally available
            # get the first year from "2019 Aston Marton DB11"
            ymm = x.find('h3', {"class" : "title-three"}).string             # 2019 Aston Marton DB11
            m = ymm.split()
            print("9: ", m[0], type(m[0]))                                          # m[0] is 1st year = 2019 is a string
            #first-year = m[0]
            # get the price for the first year
            price = y.find_next('span', {"class" : "title-four"}).text
            print ("price = ", price)
            L1 = ['Maker', 'Model', 'Year', 'Price', 'Model-url']
            L2 = [Maker, model, m[0], price, link]
            dct = dict(zip(L1, L2))
            #print (dct)
            if  i == 0 :
                model_dct_list.insert(0, dct.copy())                        # copy dct to model_dct_list[0]
                
            else : 
                model_dct_list.append(dct.copy())
            i = i + 1
    print ("\n\n")	 
    #print("1: ", model_dct_list) 
    return    model_dct_list                

# Input: model, fyear, fprice, model_url   1st year data for specific maker/model
# Output: [ {model1, year1, price1, model_url1}, {model1, year2, price2, model_url2}, ….]

def   get_maker_model_years_price (maker, model, f_year, f_price, model_url) :
    cur_price = 0
    next_year_price = 0
    r = requests.get(model_url)
    
    # create soup2 object for the model page for a specific Maker (to search recursively for <year and Link> )
    soup2 = soup( r.content, 'html.parser')
    
    # find years and corresponding links for this specific Maker/Model/First-Year/Price/Model_url of new Cars
    i = 0
    
    # convert price (f_price) from string ($xx,xxx) into integer binary representation
    pp = f_price.replace('$', '')
    pp = pp.replace(',', '')
    cur_price = int(pp)                                                 # convert string pp into integer
    next_year_price = cur_price                                         # integer binary representation
    
    # write/append first year data for a given model into tmp_dct_list[]
    tmp_dct = {}
    tmp_dct_list.clear()
    
    L1 = ['Maker', 'Model', 'Year', 'Price', 'Model-url']
    L2 = [maker, model, f_year, f_price, model_url]
    tmp_dct = dict(zip(L1, L2))
    
    tmp_dct_list.insert(0, tmp_dct.copy())              # add first year for this specific maker/model
    
    # find index in the annual_inflation_rate_list[] for this model
    if  '2020' in f_year :                                          # 2020
        i = 1                                                       # inflation starts from 2019
    elif  '2019'  in  f_year :                                      # 2019
        i = 2                                                       # inflation starts from 2018
    elif   '2018'  in f_year :                                      # 2018
        i = 3                                                       # inflation starts from 2017
    else :
        i = -1                                                      # undefined
        
    last_year = f_year[ : ]
    
    n = 0
    for  y  in  soup2.find_all("li", {"data-analytics" : True}) :   # remaining years
        if  n == 0 :
            n = n + 1
            continue
        mm_year = y.find_next('a', href=True)                       # mm_year = “/acura/ilx/2019”
        link = "https://www.kbb.com" + mm_year['href']
        year = y.find_next('a', href=True).contents                 # year = “2019”	
        print ("f_year, p_year", f_year, year)
        if year == last_year or year == "" :
            i = i + 1
            continue
        
        last_year = year[ : ]

        # estimated cur_price (binary) = int(next_year_price  / (1 + annual_inflation_rate_List[i]) )
        cur_price = int(next_year_price / (1 + annual_inflation_rate_List[i]) )	   # integer binary representation
        next_year_price = cur_price
        p1 = str(cur_price)                                         # convert binary into string
        l = len(p1) - 3                                             # l = (string length – 3)
        p2 = p1[:l] + ',' +  p1[l:]
        p = "$" + p2                                                 # p is price in string with right format
        #p = "N/A"
        if (len(tmp_dct)) :
            tmp_dct.clear()
        L1 = ['Maker', 'Model', 'Year', 'Price', 'Model-url']
        L2 = [maker, model, year, p, model_url]
        tmp_dct = dict(zip(L1, L2))         
        
        tmp_dct_list.append (tmp_dct.copy() )
        i = i + 1                                                    # index into annual inflation rate array
    return  tmp_dct_list                                             # [ {<maker: <model, year, price, model_url} ]


# input: maker/model/<year : URL> and output: maker/model/years/price/{properties dictionary}
def  get_maker_model_year_price_properties(maker, model, year, price, model_year_url) :
    prop.clear()                                                                                     # clear the dictionary
    technology = []
    entertainment = []
    tmp_dct = {}
    i = 0
    #print ("test : ", maker, model, year, price, model_year_url)
    
    r = requests.get(model_year_url)
    # create soup3 object for the top level HTML page (to search recursively for makers, models and Links)
    soup3 = soup( r.content, 'html.parser')  

    # print(soup3.find('div', {"class":"css-1wbw7cx epomrkc1"}).find('div', {"class":"stars"}).text)
    # (1-2) find out MPG and HorsePower
    mpg = False
    hp = False
    for p  in soup3.find('div', {"class":"css-1wbw7cx epomrkc1"}).find_all('div', {"class":"stars"}) :
        parent = str(p.find_parent('div').text).strip()
        #print("parent: ", parent)
        if  "Combined Fuel Economy" in   parent :
            #print("parent is: combined Fuel Economy")
            prop['MPG'] = p.text
            #print (p.text)
            mpg = True
        elif  "Horsepower"  in   parent :
            #print("parent is: ", "Horsepower")
            prop['HP'] = p.text
            hp = True
        else :
            break
    if mpg == False :
        prop['MPG'] = "N/A"
    if hp == False :
          prop['HP'] = "N/A"

    # (3-4) find out Expert Rating and Consumer rating
    i = 0
    for p  in  soup3.find_all('div', {"class":"css-x9skgx-NumericRating e149jidm1"}) :
        i = i + 1
        if i == 1 :
            if  p.text :
                value = p.text.replace('\xa0', '')
                prop['Expert-Rating'] = value
            else :
                prop['Expert-Rating'] = "N/A"
        elif (i == 2 ): 
            if p.text :
                value = p.text.replace('\xa0', '')
                prop['Consumer-Rating'] =  value
            else :
                prop['Consumer-Rating'] =  "N/A"
        else :
            break

    # (5) Avalon safety rating
    safety = soup3.find('div', {"class":"css-b7zitt e1pswftg4"})
    if safety :
        prop['Saftety-Rating'] = safety.contents[0]
    else :
        prop['Saftety-Rating'] = "N/A"
   
    # (6) Avalon Technology
    for  p   in soup3.find('ul', {"class":"css-p619jy-StyledList-List"}).find_all('span', {"class" : "css-lml4x1-ListContent e1jmyeyl2"}) :
        technology.append(p.text)

    prop['Technology'] = technology
    
    # (7) Avalon Entertainment: skip Technology
    for  p   in   soup3.find_all('ul', {"class":"css-p619jy-StyledList-List"})[1] :
        for   e   in      p.find_all('span', {"class" : "css-lml4x1-ListContent e1jmyeyl2"}) :
            entertainment.append(e.text)
            
    prop['Entertainment'] = entertainment
    #print(prop)
    
    #tmp_dct.clear()                                                             # clear the tmp_dct dictionary
    L1 = ['Maker', 'Model', 'Year', 'Price', 'property']
    L2 = [maker, model, year, price, prop]
    tmp_dct = dict(zip(L1, L2))
    return  tmp_dct                                                             # return prop dictionary <key : value>


# start of main program
# --------------------------------------------------------------
# select one Maker URL from the Maker list.   For the selected Maker get all corresponding models
# read list of dictionaries as the value of maker_dct[Maker].  Each is a URL for Maker page
# read one maker page Url from the makers_list[]

model_dct_list = get_maker_models(makers_List[0])
print("1: list of [model, fyear, fprice, model_url]:  " , model_dct_list)
print("\n\n")

# For all models: for each model first year , get the rest of the years data for that model
model_years_dct_list.clear()                           # clear model_final_dct[] before populating

# For each model: pass 1st year data and gets back the data for all years for that model
length  = len (model_dct_list)
print("model_dct_list length = ", length)

for    i    in  range (length) :
    tmp_dct.clear()
    tmp_dct = model_dct_list[i].copy()
    maker = tmp_dct.get('Maker')
    model = tmp_dct.get('Model')
    fyear = tmp_dct.get('Year')
    fprice = tmp_dct.get('Price')
    model_url = tmp_dct.get('Model-url')
    
    if i == 0 :
        model_years_dct_list.insert(0, get_maker_model_years_price(maker, model, fyear, fprice, model_url).copy()) 
    else :
        model_years_dct_list.append(get_maker_model_years_price(maker, model, fyear, fprice, model_url).copy())
            
print("2:  model_years_dct_list[] = : ",  model_years_dct_list)
print("2: length of model_years_dct_list = ", len(model_years_dct_list))
print("done with (2) \n\n")

# for each models/years get that model/year properties into model_final_dct_list[]

model_final_dct_list.clear()                      
# clear model_final_dct[] before populating 
length = len (model_years_dct_list) 
print("8: ", length)

sum = 0
ll = len(model_years_dct_list)
for i in range (ll) :
    sum  +=  len(model_years_dct_list[i])
print("ll = ", ll)
print ("sum = ", sum)

for i  in range (length):
    k = len(model_years_dct_list[i])
    tmp_dct.clear()
    tmp_dct = model_years_dct_list[i].copy()
    for j in range (k) :
        maker = tmp_dct[j].get('Maker')
        model = tmp_dct[j].get('Model')
        year = tmp_dct[j].get('Year')
        price = tmp_dct[j].get('Price')
        model_year_url = tmp_dct[j].get('Model-url')
        
        if  i  ==  0  and j == 0 :
            print("i = j = 0 : ", maker, model, year,price, model_year_url)
            model_final_dct_list.insert(0, get_maker_model_year_price_properties(maker, model, year, price, model_year_url).copy())
        else :
            print("i != j ", maker, model, year,price, model_year_url)
            model_final_dct_list.append(get_maker_model_year_price_properties(maker, model, year, price, model_year_url).copy())

print ("3: size of model_final_dct_list = ", len(model_final_dct_list))
print("3: final model_dct_list[] = : ", model_final_dct_list)
print("\n\n")

# build the data frame for a single Maker

Maker1_df = pd.DataFrame (model_final_dct_list)
print("4: ", Maker1_df)

# Save Maker1_df.csv on the computer under Adam Home page\Springboard\Makers-DFs folder
#Maker1_df.to_csv (r'C:\Users\Default\Desktop\Maker1_DataFrame.csv', index=False)
Maker1_df.to_csv (r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker1_df.csv')
# Read the csv file back as Data Frame in your runtime Cell
# Maker1_df = pd.read_csv("C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker1_df.csv")


Maker List size =  43



9:  2020 <class 'str'>
price =  $26,925
9:  2020 <class 'str'>
price =  $55,925
9:  2020 <class 'str'>
price =  $62,925
9:  2020 <class 'str'>
price =  $34,025
9:  2020 <class 'str'>
price =  $45,525
9:  2020 <class 'str'>
price =  $54,025
9:  2020 <class 'str'>
price =  $38,825
9:  2019 <class 'str'>
price =  $159,300



1: list of [model, fyear, fprice, model_url]:   [{'Maker': 'acura', 'Model': 'ilx', 'Year': '2020', 'Price': '$26,925', 'Model-url': 'https://www.kbb.com/acura/ilx/'}, {'Maker': 'acura', 'Model': 'rlx', 'Year': '2020', 'Price': '$55,925', 'Model-url': 'https://www.kbb.com/acura/rlx/'}, {'Maker': 'acura', 'Model': 'rlx-sport-hybrid', 'Year': '2020', 'Price': '$62,925', 'Model-url': 'https://www.kbb.com/acura/rlx-sport-hybrid/'}, {'Maker': 'acura', 'Model': 'tlx', 'Year': '2020', 'Price': '$34,025', 'Model-url': 'https://www.kbb.com/acura/tlx/'}, {'Maker': 'acura', 'Model': 'mdx', 'Year': '2020', 'Price': '$45,525', 'Model-url': 'https://www.kbb

In [2]:
from     bs4 import  BeautifulSoup as soup
import   requests
import   pandas as pd
from     urllib.request  import  urlopen

# Bring CSV files (e.g., Maker1_df.csv) into memory as Data Frame objects (e.g., Maker1_df):
Maker1_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker1_df.csv')
Maker2_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker2_df.csv')
Maker3_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker3_df.csv')
Maker4_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker4_df.csv')
Maker5_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker5_df.csv')
Maker6_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker6_df.csv')
Maker7_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker7_df.csv')
Maker9_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker9_df.csv')
Maker10_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker10_df.csv')
Maker11_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker11_df.csv')
Maker12_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker12_df.csv')
Maker13_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker13_df.csv')
Maker14_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker14_df.csv')
Maker15_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker15_df.csv')
Maker16_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker16_df.csv')
Maker17_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker17_df.csv')
Maker18_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker18_df.csv')
Maker19_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker19_df.csv')
Maker20_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker20_df.csv')
Maker21_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker21_df.csv')
Maker22_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker22_df.csv')
Maker23_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker23_df.csv')
Maker24_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker24_df.csv')
Maker25_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker25_df.csv')
Maker26_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker26_df.csv')
Maker27_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker27_df.csv')
Maker28_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker28_df.csv')
Maker29_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker29_df.csv')
Maker30_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker30_df.csv')
Maker32_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker32_df.csv')
Maker33_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker33_df.csv')
Maker34_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker34_df.csv')
Maker35_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker35_df.csv')
Maker36_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker36_df.csv')
Maker37_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker37_df.csv')
Maker38_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker38_df.csv')
Maker39_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker39_df.csv')
Maker40_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker40_df.csv')
Maker41_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker41_df.csv')
Maker42_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker42_df.csv')
Maker43_df = pd.read_csv(r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker43_df.csv')

# Concatenate these Data Frame files into one Data Frame
df = [ Maker1_df, Maker2_df, Maker3_df, Maker4_df, Maker5_df, Maker6_df, Maker7_df, Maker9_df, Maker10_df, Maker11_df, Maker12_df, Maker13_df, Maker14_df, Maker15_df, Maker16_df, Maker17_df, Maker18_df, Maker19_df, Maker20_df, Maker21_df, Maker22_df, Maker23_df, Maker24_df, Maker25_df, Maker26_df, Maker27_df, Maker28_df, Maker29_df, Maker30_df, Maker32_df, Maker33_df, Maker34_df, Maker35_df, Maker36_df, Maker37_df, Maker38_df, Maker39_df, Maker40_df, Maker41_df, Maker42_df, Maker43_df ] 

# Store this single Data Frame file as CSV file on local disk
Global_df = pd.concat(df)

# Bring CSV files (e.g., Maker1_df.csv) into memory as Data Frame objects (e.g., Maker1_df):
Global_df.to_csv (r'C:\Users\AdamEzzat\Springboard\Makers-DFs\Global_df.csv')




In [3]:
import ast
sample_dict = ast.literal_eval(Global_df.head()['property'].iloc[0])
print(sample_dict)

{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating': '0.0', 'Saftety-Rating': 'N/A', 'Technology': ['Bluetooth Wireless Technology', 'Cruise Control', 'Hands Free Phone', 'Navigation System', 'Remote Keyless Entry', 'Audible Text Messaging', 'Proximity Sensing Keyless Entry', 'Push-Button Engine Start', 'Real-Time Traffic Information', 'Smartphone Interface', 'USB Port', 'Voice Recognition System'], 'Entertainment': ['MP3 Player', 'Premium Radio', 'Satellite Radio', 'Bluetooth Streaming Audio']}


In [4]:
Global_df['property_dict']= Global_df['property'].apply(lambda x: ast.literal_eval(x))

In [5]:
for key in sample_dict: 
    Global_df[key] = Global_df['property_dict'].apply(lambda x: x[key] if key in x else "ERROR")

In [6]:
Global_df

,Unnamed: 0,Maker,Model,Year,Price,property,property_dict,MPG,HP,Expert-Rating,Saftety-Rating,Technology,Entertainment
0,0,acura,ilx,2020,"$26,925","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...",21 MPG,N/A,0.0,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[MP3 Player, Premium Radio, Satellite Radio, B..."
1,1,acura,ilx,['2019'],"$26,448","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...",21 MPG,N/A,0.0,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[MP3 Player, Premium Radio, Satellite Radio, B..."
2,2,acura,ilx,['2018'],"$25,828","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...",21 MPG,N/A,0.0,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[MP3 Player, Premium Radio, Satellite Radio, B..."
3,3,acura,ilx,['2017'],"$25,296","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...",21 MPG,N/A,0.0,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[MP3 Player, Premium Radio, Satellite Radio, B..."
4,4,acura,ilx,['2016'],"$24,971","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...",21 MPG,N/A,0.0,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[MP3 Player, Premium Radio, Satellite Radio, B..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,62,volvo,v90,['2019'],"$51,517","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...",26 MPG,250 HP,4.3,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[Premium Radio, Satellite Radio, Bluetooth Str..."
63,63,volvo,v90,['2018'],"$50,309","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...",26 MPG,250 HP,4.3,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[Premium Radio, Satellite Radio, Bluetooth Str..."
64,64,volvo,v90,['2017'],"$49,274","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...",26 MPG,250 HP,4.3,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[Premium Radio, Satellite Radio, Bluetooth Str..."
65,65,volvo,v90,['1998'],"$48,641","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...",26 MPG,250 HP,4.3,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[Premium Radio, Satellite Radio, Bluetooth Str..."


In [7]:
all_tech = set(Global_df['Technology'].sum())

In [8]:
for tech in all_tech: 
    Global_df[tech] = Global_df['Technology'].apply(lambda x: 1 if tech in x else 0)

In [9]:
Global_df

,Unnamed: 0,Maker,Model,Year,Price,property,property_dict,MPG,HP,Expert-Rating,...,Real-Time Traffic Information,Internet Access,Remote Engine Start,Number of Doors: 4 doors,Touch Screen Monitor,Bluetooth Wireless Technology,Push-Button Engine Start,Premium Radio,Proximity Sensing Keyless Entry,Navigation System
0,0,acura,ilx,2020,"$26,925","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...",21 MPG,N/A,0.0,...,1,0,0,0,0,1,1,0,1,1
1,1,acura,ilx,['2019'],"$26,448","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...",21 MPG,N/A,0.0,...,1,0,0,0,0,1,1,0,1,1
2,2,acura,ilx,['2018'],"$25,828","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...",21 MPG,N/A,0.0,...,1,0,0,0,0,1,1,0,1,1
3,3,acura,ilx,['2017'],"$25,296","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...",21 MPG,N/A,0.0,...,1,0,0,0,0,1,1,0,1,1
4,4,acura,ilx,['2016'],"$24,971","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...",21 MPG,N/A,0.0,...,1,0,0,0,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,62,volvo,v90,['2019'],"$51,517","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...",26 MPG,250 HP,4.3,...,0,1,1,0,0,1,1,0,1,1
63,63,volvo,v90,['2018'],"$50,309","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...",26 MPG,250 HP,4.3,...,0,1,1,0,0,1,1,0,1,1
64,64,volvo,v90,['2017'],"$49,274","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...",26 MPG,250 HP,4.3,...,0,1,1,0,0,1,1,0,1,1
65,65,volvo,v90,['1998'],"$48,641","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...",26 MPG,250 HP,4.3,...,0,1,1,0,0,1,1,0,1,1


In [27]:
all_ent = set(Global_df['Entertainment'].sum())

In [28]:
for ent in all_ent: 
    Global_df[ent] = Global_df['Entertainment'].apply(lambda x: 1 if ent in x else 0)

In [29]:
Global_df

,Unnamed: 0,Maker,Model,Year,Price,property,property_dict,MPG,HP,Expert-Rating,...,Leather Seats,Bluetooth Streaming Audio,Power Driver's Seat,Heated Seats,Third Row Seat,MP3 Player,iPod Connector,Cloth Seats,Dual Power Front Seats,Satellite Radio
0,0,acura,ilx,2020,"$26,925","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...",21 MPG,N/A,0.0,...,0,1,0,0,0,1,0,0,0,1
1,1,acura,ilx,['2019'],"$26,448","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...",21 MPG,N/A,0.0,...,0,1,0,0,0,1,0,0,0,1
2,2,acura,ilx,['2018'],"$25,828","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...",21 MPG,N/A,0.0,...,0,1,0,0,0,1,0,0,0,1
3,3,acura,ilx,['2017'],"$25,296","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...",21 MPG,N/A,0.0,...,0,1,0,0,0,1,0,0,0,1
4,4,acura,ilx,['2016'],"$24,971","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...",21 MPG,N/A,0.0,...,0,1,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,62,volvo,v90,['2019'],"$51,517","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...",26 MPG,250 HP,4.3,...,0,1,0,0,0,0,0,0,0,1
63,63,volvo,v90,['2018'],"$50,309","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...",26 MPG,250 HP,4.3,...,0,1,0,0,0,0,0,0,0,1
64,64,volvo,v90,['2017'],"$49,274","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...",26 MPG,250 HP,4.3,...,0,1,0,0,0,0,0,0,0,1
65,65,volvo,v90,['1998'],"$48,641","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...",26 MPG,250 HP,4.3,...,0,1,0,0,0,0,0,0,0,1


In [10]:
print(Global_df.head)

<bound method NDFrame.head of     Unnamed: 0  Maker Model      Year    Price  \
0            0  acura   ilx      2020  $26,925   
1            1  acura   ilx  ['2019']  $26,448   
2            2  acura   ilx  ['2018']  $25,828   
3            3  acura   ilx  ['2017']  $25,296   
4            4  acura   ilx  ['2016']  $24,971   
..         ...    ...   ...       ...      ...   
62          62  volvo   v90  ['2019']  $51,517   
63          63  volvo   v90  ['2018']  $50,309   
64          64  volvo   v90  ['2017']  $49,274   
65          65  volvo   v90  ['1998']  $48,641   
66          66  volvo   v90  ['1997']  $48,592   

                                             property  \
0   {'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...   
1   {'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...   
2   {'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...   
3   {'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...   
4   {'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...   
..                         

In [11]:
pd.set_option('display.max_columns', None)
Global_df.head()

,Unnamed: 0,Maker,Model,Year,Price,property,property_dict,MPG,HP,Expert-Rating,Saftety-Rating,Technology,Entertainment,Remote Keyless Entry,Hands Free Phone,Cruise Control,Audible Text Messaging,USB Port,Smartphone Interface,Voice Recognition System,Real-Time Traffic Information,Internet Access,Remote Engine Start,Number of Doors: 4 doors,Touch Screen Monitor,Bluetooth Wireless Technology,Push-Button Engine Start,Premium Radio,Proximity Sensing Keyless Entry,Navigation System
0,0,acura,ilx,2020,"$26,925","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...",21 MPG,N/A,0.0,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[MP3 Player, Premium Radio, Satellite Radio, B...",1,1,1,1,1,1,1,1,0,0,0,0,1,1,0,1,1
1,1,acura,ilx,['2019'],"$26,448","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...",21 MPG,N/A,0.0,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[MP3 Player, Premium Radio, Satellite Radio, B...",1,1,1,1,1,1,1,1,0,0,0,0,1,1,0,1,1
2,2,acura,ilx,['2018'],"$25,828","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...",21 MPG,N/A,0.0,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[MP3 Player, Premium Radio, Satellite Radio, B...",1,1,1,1,1,1,1,1,0,0,0,0,1,1,0,1,1
3,3,acura,ilx,['2017'],"$25,296","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...",21 MPG,N/A,0.0,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[MP3 Player, Premium Radio, Satellite Radio, B...",1,1,1,1,1,1,1,1,0,0,0,0,1,1,0,1,1
4,4,acura,ilx,['2016'],"$24,971","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...",21 MPG,N/A,0.0,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[MP3 Player, Premium Radio, Satellite Radio, B...",1,1,1,1,1,1,1,1,0,0,0,0,1,1,0,1,1


In [12]:
import pandas as pd
Car_Data = pd.read_csv('Car_Data.csv' , engine = 'python')
print(Car_Data.head(5))

    Maker Condition    Fuel  KMs Driven        Model    Price Registered City  \
0  Toyota      Used  Diesel         1.0        Prado  2100000         Karachi   
1  Suzuki      Used  Petrol    100000.0        Bolan   380000         Karachi   
2  Suzuki      Used     CNG     12345.0        Bolan   340000         Karachi   
3  Suzuki      Used  Petrol     94000.0         Alto   535000         Karachi   
4  Toyota      Used  Petrol    100000.0  Corolla XLI  1430000         Karachi   

  Transaction Type    Year  
0             Cash  1997.0  
1             Cash  2006.0  
2             Cash  1998.0  
3             Cash  2010.0  
4             Cash  2013.0  


In [13]:
Global_df.dtypes

Unnamed: 0                          int64
Maker                              object
Model                              object
Year                               object
Price                              object
property                           object
property_dict                      object
MPG                                object
HP                                 object
Expert-Rating                      object
Saftety-Rating                     object
Technology                         object
Entertainment                      object
Remote Keyless Entry                int64
Hands Free Phone                    int64
Cruise Control                      int64
Audible Text Messaging              int64
USB Port                            int64
Smartphone Interface                int64
Voice Recognition System            int64
Real-Time Traffic Information       int64
Internet Access                     int64
Remote Engine Start                 int64
Number of Doors: 4 doors          

In [33]:
Global_df.rename(columns={"Price": "New_Price",})

,Unnamed: 0,Maker,Model,Year,New_Price,property,property_dict,MPG,HP,Expert-Rating,Saftety-Rating,Technology,Entertainment,Remote Keyless Entry,Hands Free Phone,Cruise Control,Audible Text Messaging,USB Port,Smartphone Interface,Voice Recognition System,Real-Time Traffic Information,Internet Access,Remote Engine Start,Number of Doors: 4 doors,Touch Screen Monitor,Bluetooth Wireless Technology,Push-Button Engine Start,Premium Radio,Proximity Sensing Keyless Entry,Navigation System
0,0,acura,ilx,2020,"$26,925","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...",21 MPG,N/A,0.0,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[MP3 Player, Premium Radio, Satellite Radio, B...",1,1,1,1,1,1,1,1,0,0,0,0,1,1,0,1,1
1,1,acura,ilx,['2019'],"$26,448","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...",21 MPG,N/A,0.0,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[MP3 Player, Premium Radio, Satellite Radio, B...",1,1,1,1,1,1,1,1,0,0,0,0,1,1,0,1,1
2,2,acura,ilx,['2018'],"$25,828","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...",21 MPG,N/A,0.0,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[MP3 Player, Premium Radio, Satellite Radio, B...",1,1,1,1,1,1,1,1,0,0,0,0,1,1,0,1,1
3,3,acura,ilx,['2017'],"$25,296","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...",21 MPG,N/A,0.0,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[MP3 Player, Premium Radio, Satellite Radio, B...",1,1,1,1,1,1,1,1,0,0,0,0,1,1,0,1,1
4,4,acura,ilx,['2016'],"$24,971","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...",21 MPG,N/A,0.0,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[MP3 Player, Premium Radio, Satellite Radio, B...",1,1,1,1,1,1,1,1,0,0,0,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,62,volvo,v90,['2019'],"$51,517","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...",26 MPG,250 HP,4.3,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[Premium Radio, Satellite Radio, Bluetooth Str...",1,1,1,0,1,1,1,0,1,1,0,0,1,1,0,1,1
63,63,volvo,v90,['2018'],"$50,309","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...",26 MPG,250 HP,4.3,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[Premium Radio, Satellite Radio, Bluetooth Str...",1,1,1,0,1,1,1,0,1,1,0,0,1,1,0,1,1
64,64,volvo,v90,['2017'],"$49,274","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...",26 MPG,250 HP,4.3,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[Premium Radio, Satellite Radio, Bluetooth Str...",1,1,1,0,1,1,1,0,1,1,0,0,1,1,0,1,1
65,65,volvo,v90,['1998'],"$48,641","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...",26 MPG,250 HP,4.3,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[Premium Radio, Satellite Radio, Bluetooth Str...",1,1,1,0,1,1,1,0,1,1,0,0,1,1,0,1,1


In [34]:
Global_df.columns

Index(['Unnamed: 0', 'Maker', 'Model', 'Year', 'Price', 'property',
       'property_dict', 'MPG', 'HP', 'Expert-Rating', 'Saftety-Rating',
       'Technology', 'Entertainment', 'Remote Keyless Entry',
       'Hands Free Phone', 'Cruise Control', 'Audible Text Messaging',
       'USB Port', 'Smartphone Interface', 'Voice Recognition System',
       'Real-Time Traffic Information', 'Internet Access',
       'Remote Engine Start', 'Number of Doors: 4 doors',
       'Touch Screen Monitor', 'Bluetooth Wireless Technology',
       'Push-Button Engine Start', 'Premium Radio',
       'Proximity Sensing Keyless Entry', 'Navigation System'],
      dtype='object')

In [25]:
Car_Data.dtypes

Maker                object
Condition            object
Fuel                 object
KMs Driven          float64
Model                object
Price                 int64
Registered City      object
Transaction Type     object
Year                 object
dtype: object

In [26]:
import pandas as pd
convert_dict = {'Year': object} 
  
Car_Data = Car_Data.astype(convert_dict) 
print(Car_Data.dtypes) 

Maker                object
Condition            object
Fuel                 object
KMs Driven          float64
Model                object
Price                 int64
Registered City      object
Transaction Type     object
Year                 object
dtype: object


In [22]:
print(Car_Data)

          Maker Condition    Fuel  KMs Driven        Model    Price  \
0        Toyota      Used  Diesel         1.0        Prado  2100000   
1        Suzuki      Used  Petrol    100000.0        Bolan   380000   
2        Suzuki      Used     CNG     12345.0        Bolan   340000   
3        Suzuki      Used  Petrol     94000.0         Alto   535000   
4        Toyota      Used  Petrol    100000.0  Corolla XLI  1430000   
...         ...       ...     ...         ...          ...      ...   
22541    Toyota      Used  Petrol     76190.0       Avanza  1175000   
22542    Toyota      Used     CNG    200000.0   Corolla XE  1070000   
22543  Daihatsu       New  Petrol     10000.0        Cuore   390000   
22544    Suzuki      Used  Petrol         1.0         Alto   470000   
22545    Toyota      Used  Petrol     48500.0  Corolla GLI  2050000   

      Registered City Transaction Type  Year  
0             Karachi             Cash  1997  
1             Karachi             Cash  2006  
2     

In [27]:
Global_df.merge(Car_Data, left_on = Global_df.keys(), right_on = Car_Data.keys(), how ='inner')

KeyError: Index(['Maker', 'Condition', 'Fuel', 'KMs Driven', 'Model', 'Price',
       'Registered City', 'Transaction Type', 'Year'],
      dtype='object')

In [30]:
Global_df

,Unnamed: 0,Maker,Model,Year,Price,property,property_dict,MPG,HP,Expert-Rating,Saftety-Rating,Technology,Entertainment,Remote Keyless Entry,Hands Free Phone,Cruise Control,Audible Text Messaging,USB Port,Smartphone Interface,Voice Recognition System,Real-Time Traffic Information,Internet Access,Remote Engine Start,Number of Doors: 4 doors,Touch Screen Monitor,Bluetooth Wireless Technology,Push-Button Engine Start,Premium Radio,Proximity Sensing Keyless Entry,Navigation System
0,0,acura,ilx,2020,"$26,925","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...",21 MPG,N/A,0.0,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[MP3 Player, Premium Radio, Satellite Radio, B...",1,1,1,1,1,1,1,1,0,0,0,0,1,1,0,1,1
1,1,acura,ilx,['2019'],"$26,448","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...",21 MPG,N/A,0.0,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[MP3 Player, Premium Radio, Satellite Radio, B...",1,1,1,1,1,1,1,1,0,0,0,0,1,1,0,1,1
2,2,acura,ilx,['2018'],"$25,828","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...",21 MPG,N/A,0.0,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[MP3 Player, Premium Radio, Satellite Radio, B...",1,1,1,1,1,1,1,1,0,0,0,0,1,1,0,1,1
3,3,acura,ilx,['2017'],"$25,296","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...",21 MPG,N/A,0.0,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[MP3 Player, Premium Radio, Satellite Radio, B...",1,1,1,1,1,1,1,1,0,0,0,0,1,1,0,1,1
4,4,acura,ilx,['2016'],"$24,971","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...","{'MPG': '21 MPG', 'HP': 'N/A', 'Expert-Rating'...",21 MPG,N/A,0.0,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[MP3 Player, Premium Radio, Satellite Radio, B...",1,1,1,1,1,1,1,1,0,0,0,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,62,volvo,v90,['2019'],"$51,517","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...",26 MPG,250 HP,4.3,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[Premium Radio, Satellite Radio, Bluetooth Str...",1,1,1,0,1,1,1,0,1,1,0,0,1,1,0,1,1
63,63,volvo,v90,['2018'],"$50,309","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...",26 MPG,250 HP,4.3,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[Premium Radio, Satellite Radio, Bluetooth Str...",1,1,1,0,1,1,1,0,1,1,0,0,1,1,0,1,1
64,64,volvo,v90,['2017'],"$49,274","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...",26 MPG,250 HP,4.3,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[Premium Radio, Satellite Radio, Bluetooth Str...",1,1,1,0,1,1,1,0,1,1,0,0,1,1,0,1,1
65,65,volvo,v90,['1998'],"$48,641","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...","{'MPG': '26 MPG', 'HP': '250 HP', 'Expert-Rati...",26 MPG,250 HP,4.3,N/A,"[Bluetooth Wireless Technology, Cruise Control...","[Premium Radio, Satellite Radio, Bluetooth Str...",1,1,1,0,1,1,1,0,1,1,0,0,1,1,0,1,1
